In [29]:
#
# Please execute on QITE_experiments branch from GitHub
# 

from qrisp import QuantumVariable, h, x, prepare, conjugate
from qrisp.vqe import VQEProblem
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function, heisenberg_problem, create_heisenberg_hamiltonian
import networkx as nx
import numpy as np
import scipy
import matplotlib.pyplot as plt
from run_QITE_PRL import run_QITE_PRL

import pickle
from time import time
from datetime import datetime

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

In [30]:
L = 20
G = nx.Graph()
G.add_edges_from([(k,(k+1)%L) for k in range(L-1)]) 
# nx.draw(G, with_labels=True)

In [ ]:
from qrisp.operators import X, Y, Z

J = 1
B = 0
H = create_heisenberg_hamiltonian(G, J, B)
print(H)

# Define scaling factor
F = 10

# Hamiltonian simulation via first order Suzuki-Trotter formula with 5 steps
def exp_H(qv, t):
    H.trotterization(method='commuting')(qv,t/F,5)

# Hamiltonian simulation via second order Suzuki-Trotter formula with 2 steps
def exp_H_2(qv, t):
    H.trotterization(order=2,method='commuting')(qv,t/F,2)

X(0)*X(1) + X(1)*X(2) + X(10)*X(11) + X(11)*X(12) + X(12)*X(13) + X(13)*X(14) + X(14)*X(15) + X(15)*X(16) + X(16)*X(17) + X(17)*X(18) + X(18)*X(19) + X(2)*X(3) + X(3)*X(4) + X(4)*X(5) + X(5)*X(6) + X(6)*X(7) + X(7)*X(8) + X(8)*X(9) + X(9)*X(10) + Y(0)*Y(1) + Y(1)*Y(2) + Y(10)*Y(11) + Y(11)*Y(12) + Y(12)*Y(13) + Y(13)*Y(14) + Y(14)*Y(15) + Y(15)*Y(16) + Y(16)*Y(17) + Y(17)*Y(18) + Y(18)*Y(19) + Y(2)*Y(3) + Y(3)*Y(4) + Y(4)*Y(5) + Y(5)*Y(6) + Y(6)*Y(7) + Y(7)*Y(8) + Y(8)*Y(9) + Y(9)*Y(10) + Z(0)*Z(1) + Z(1)*Z(2) + Z(10)*Z(11) + Z(11)*Z(12) + Z(12)*Z(13) + Z(13)*Z(14) + Z(14)*Z(15) + Z(15)*Z(16) + Z(16)*Z(17) + Z(17)*Z(18) + Z(18)*Z(19) + Z(2)*Z(3) + Z(3)*Z(4) + Z(4)*Z(5) + Z(5)*Z(6) + Z(6)*Z(7) + Z(7)*Z(8) + Z(8)*Z(9) + Z(9)*Z(10)


## QITE with singlet, plus, vqe warm start

In [32]:
# Tensor product of singlet states
M = nx.maximal_matching(G)
U_singlet = create_heisenberg_init_function(M)

# HVA
H0 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in M)
H1 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in set(G.edges())-M)

def ansatz(qv,theta):
    # Rescaling t->1/4 equivalent to choosing rescaled inital parameters for VQE
    H1.trotterization(method='commuting')(qv,t=theta[1]/4)
    H0.trotterization(method='commuting')(qv,t=theta[0]/4)

#vqe = heisenberg_problem(G,J,B, ansatz_type='per hamiltonian')
vqe = VQEProblem(H, ansatz, 2, init_function=U_singlet)
U_vqe = vqe.train_function(QuantumVariable(G.number_of_nodes()),depth=1,max_iter=100)

# Calculate energy for HVA
def state_prep():
    qv = QuantumVariable(L)
    U_vqe(qv)
    return qv

print(H.expectation_value(state_prep)())

results = []

results.append(run_QITE_PRL(H, U_singlet, exp_H_2, np.linspace(.01,1.5,20), 5, method='GC', use_statevectors=True))
results.append(run_QITE_PRL(H, U_vqe, exp_H_2, np.linspace(.01,1.5,20), 5, method='GC', use_statevectors=True))

-33.83877340137857                                                                   


In [ ]:
# Save benchmark data
current_datetime = datetime.now()
date_time_string = current_datetime.strftime("%m-%d-%H")
save_data(results,'data/benchmarks_2nd_L='+str(L)+'_'+date_time_string)